### Regularyzacja przez mnożniki Lagrange'a. Algorytm SVM


#### 1. Zadanie 1 (2pkt):

Rozwiń algorytm regresji logistycznej z lab. 1, wprowadzając do niego człon regularyzacyjny


##### Methods definition


In [4]:
import sys
import subprocess
import pkg_resources
import numpy as np
import os
from skimage.io import imread
import cv2 as cv
from pathlib import Path
import json


required = {"scikit-image"}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call(
        [python, "-m", "pip", "install", *missing], stdout=subprocess.DEVNULL
    )


def load_train_data(input_dir, newSize=(64, 64)):
    image_dir = Path(input_dir)
    categories_name = []
    for file in os.listdir(image_dir):
        d = os.path.join(image_dir, file)
        if os.path.isdir(d):
            categories_name.append(file)

    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]

    train_img = []
    categories_count = []
    labels = []
    for _, direc in enumerate(folders):
        count = 0
        for obj in direc.iterdir():
            if (
                os.path.isfile(obj)
                and os.path.basename(os.path.normpath(obj)) != "desktop.ini"
            ):
                labels.append(os.path.basename(os.path.normpath(direc)))
                count += 1
                img = imread(obj)  # zwraca ndarry postaci xSize x ySize x colorDepth
                img = cv.resize(
                    img, newSize, interpolation=cv.INTER_AREA
                )  # zwraca ndarray
                if img[0][0].size==3:
                    img =  np.dstack((img,np.ones((64,64,1))))
                img = img / 255  # normalizacja
                train_img.append(img)
        categories_count.append(count)
    X = {}
    # return train_img
    X["values"] = np.array(train_img)
    X["categories_name"] = categories_name
    X["categories_count"] = categories_count
    X["labels"] = labels
    return X


def load_test_data(input_dir, newSize=(64, 64)):
    image_path = Path(input_dir)
    labels_path = image_path.parents[0] / "test_labels.json"

    # with labels_path.open("r", encoding ="utf-8") as f:
    jsonString = labels_path.read_text()
    objects = json.loads(jsonString)

    categories_name = []
    categories_count = []
    count = 0
    c = objects[0]["value"]
    for e in objects:
        if e["value"] != c:
            # print(count)
            # print(c)
            categories_count.append(count)
            c = e["value"]
            count = 1
        else:
            count += 1
        if not e["value"] in categories_name:
            categories_name.append(e["value"])
    categories_count.append(count)

    test_img = []
    labels = []

    for e in objects:
        p = image_path / e["filename"]
        img = imread(p)  # zwraca ndarry postaci xSize x ySize x colorDepth
        img = cv.resize(img, newSize, interpolation=cv.INTER_AREA)  # zwraca ndarray
        img = img / 255  # normalizacja
        test_img.append(img)
        labels.append(e["value"])

    X = {}
    X["values"] = np.array(test_img)
    X["categories_name"] = categories_name
    X["categories_count"] = categories_count
    X["labels"] = labels
    return X


##### Data loading


In [5]:
from sklearn.preprocessing import LabelEncoder


data_train = load_train_data("./train_test_sw/train_sw", newSize=(64, 64))
X_train = data_train["values"]
y_train = data_train["labels"]

data_test = load_test_data("./train_test_sw/test_sw", newSize=(64, 64))
X_test = data_test["values"]
y_test = data_test["labels"]

class_le = LabelEncoder()
y_train_enc = class_le.fit_transform(y_train)
y_test_enc = class_le.fit_transform(y_test)


##### LogReg class


In [3]:
class LogisticRegression:
    def __init__(self, data_shape, classes_no) -> None:
        self.w = np.zeros(shape=(classes_no, data_shape[0], 1))
        self.b = np.zeros(shape=(classes_no, 1))
        print(f"self.w.shape = {self.w.shape}\nself.b.shape = {self.b.shape}")

    def sigmoid(self, z) -> float:
        return 1 / (1 + np.exp(-z)) - 0.0000001

    def propagate(self, w, b, X, Y, regularization, lambda_param) -> tuple:
        data_len = X.shape[1]

        A = self.sigmoid((np.dot(w.T, X) + b))

        # ==========================
        # Regularization part
        reg_val = lambda_param * np.sum(w.T) if regularization else 0
        cost = (
            np.sum(((-np.log(A)) * Y + (-np.log(1 - A)) * (1 - Y))) - reg_val
        ) / data_len
        # ==========================

        cost = np.squeeze(cost)

        dw = (np.dot(X, (A - Y).T)) / data_len
        db = (np.sum(A - Y)) / data_len

        grads = {"dw": dw, "db": db}
        return grads, cost

    def fit(
        self,
        X,
        Y,
        num_iterations,
        lr,
        verbose=False,
        regularization=None,
        lambda_param=0.1,
    ) -> dict:
        error_vals = [[] for i in range(self.w.shape[0])]
        for i in range(num_iterations):
            if (i + 1) % 100 == 1 or i == num_iterations - 1:
                print("Iteration:", i)
            for ent_class in range(self.w.shape[0]):

                grads, cost = self.propagate(
                    self.w[ent_class],
                    self.b[ent_class],
                    X,
                    Y[ent_class],
                    regularization,
                    lambda_param,
                )
                # print(f"\t{i}before w update")
                self.w[ent_class] -= lr * grads["dw"]
                # print("\tbefore b update")
                self.b[ent_class] -= lr * grads["db"]
                if (i + 1) % 100 == 1 or i == num_iterations - 1:
                    error_vals[ent_class].append(cost)
                    if verbose:
                        print(f"\tClass: {ent_class} → error: {cost}")
        return {
            "parameters": {"w": self.w, "b": self.b},
            "gradients": grads,
            "error_values": error_vals,
        }

    def predict(self, X) -> list:
        A = np.array(
            [
                self.sigmoid(np.dot(current_w.reshape(X.shape[0], -1).T, X) + current_b)
                for current_w, current_b in zip(self.w, self.b)
            ]
        )
        Y_pred = [(class_preds > 0.5) * 1.0 for class_preds in A]  # A * 5.0
        return Y_pred

    def validate(self, test_data, true_labels) -> str:
        preds = self.predict(test_data)
        return f"Accuracy for the given data → {100 - np.mean(np.abs(preds - true_labels))*100}"


def prepare_data(x_data, y_data) -> tuple:
    x = x_data.reshape(x_data.shape[0], -1).T
    y = np.array([[int(x == i) for i in y_data] for x in range(len(set(y_data)))])
    return x, y


##### Fitting


In [4]:
X_train_flat, y_train_enc_multi = prepare_data(X_train, y_train_enc)

# Logistic Regression training
log_reg = LogisticRegression(X_train_flat.shape, classes_no=5)
training_result = log_reg.fit(
    X_train_flat,
    y_train_enc_multi,
    num_iterations=2000,
    lr=0.0029,
    verbose=True,
    regularization=True,
)


self.w.shape = (5, 16384, 1)
self.b.shape = (5, 1)
Iteration: 0
	Class: 0 → error: 0.6931470596252036
	Class: 1 → error: 0.6931470607936555
	Class: 2 → error: 0.6931470607936555
	Class: 3 → error: 0.6931470607936555
	Class: 4 → error: 0.6931470607936555
Iteration: 100
	Class: 0 → error: 0.4267814228420924
	Class: 1 → error: 0.49106950859658366
	Class: 2 → error: 0.48079657435762524
	Class: 3 → error: 0.483711936075606
	Class: 4 → error: 0.4800875219526501
Iteration: 200
	Class: 0 → error: 0.3907069724129813
	Class: 1 → error: 0.48388959897515377
	Class: 2 → error: 0.4644157832460014
	Class: 3 → error: 0.4716988962252928
	Class: 4 → error: 0.46848768835863797
Iteration: 300
	Class: 0 → error: 0.3684083064604981
	Class: 1 → error: 0.478110817242968
	Class: 2 → error: 0.4504614181247573
	Class: 3 → error: 0.4623617531091419
	Class: 4 → error: 0.46119779803927824
Iteration: 400
	Class: 0 → error: 0.3524730223063941
	Class: 1 → error: 0.4730996505359099
	Class: 2 → error: 0.4381849580583522

##### Validation


In [5]:
X_test_flat, y_test_enc_multi = prepare_data(X_test, y_test_enc)

preds = log_reg.predict(X_test_flat)
size = np.size(preds - y_test_enc_multi)
fp = np.sum(np.abs(preds - y_test_enc_multi))
tp = size - fp
fn = np.count_nonzero((preds - y_test_enc_multi) > 0)

print(
    f"Accuracy for the given data → {100 - np.mean(np.abs(preds - y_test_enc_multi))*100}%"
)
print(f"Precision for the given data → { tp/(tp+fp) }")
print(f"Recall for the given data → { tp/(tp+fn) }")
print(f"F1 score for the given data → { 2*tp / (2*tp + fp + fn) }")


# print(preds)


Accuracy for the given data → 77.82239382239382%
Precision for the given data → 0.7782239382239382
Recall for the given data → 0.9640329060646643
F1 score for the given data → 0.8612203042215006


#### Zadanie 2 (4pkt)

Zaimplementuj algorytm SVM z miękkim marginesem (regularyzacją)


In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np


In [10]:
class SVM:
    def __init__(self, x_train,y_train, classes_no,lambda_param=0.001) -> None:
        self.x_train = np.array([x.flatten() for x in x_train])
        self.y_train = y_train
        self.lambda_param = lambda_param
        self.w = np.zeros(shape=(self.x_train.shape[1],classes_no))
        self.b = np.zeros(shape=(1,classes_no))
        print(f"self.w.shape = {self.w.shape}\nself.b.shape = {self.b.shape}")

    def cost(self) -> tuple:
        delta = 0.5
        scores = np.dot(self.x_train,self.w) 
        correct_class_score = scores[np.arange(self.x_train.shape[0]), self.y_train]
        margins = np.maximum(0, scores - correct_class_score[:,np.newaxis] + delta)
        margins[np.arange(self.x_train.shape[0]), self.y_train] = 0
        loss = np.sum(margins)/ self.x_train.shape[0]
        loss +=   0.5*self.lambda_param* np.sum(self.w * self.w)
        return loss, margins

    def propagate(self) -> tuple:
        loss,margins = self.cost()
        error = np.zeros(margins.shape)
        error[margins > 0] = 1 
        count = np.sum(error,axis=1)
        error[np.arange(self.x_train.shape[0]),self.y_train] = -count

        dw = self.x_train.T.dot(error)/self.x_train.shape[0]

        # Regularize
        dw += 2*self.lambda_param*self.w
        db = np.sum(error, axis=0) / self.x_train.shape[0]
        db.reshape(1, 5)

        grads = {"dw": dw, "db": db}
        return grads, loss

    def fit(
        self,
        num_iterations,
        lr,
        verbose=False,
    ) -> dict:
        error_vals = [[] for i in range(self.w.shape[0])]
        for i in range(num_iterations):
            if (i + 1) % 100 == 1 or i == num_iterations - 1:
                print("Iteration:", i)
            for ent_class in range(self.w.shape[1]):

                grads, cost = self.propagate()
                # print(f"\t{i}before w update")
                self.w -= lr * grads["dw"]
                # print("\tbefore b update")
                self.b -= lr * grads["db"]
                if (i + 1) % 100 == 1 or i == num_iterations - 1:
                    error_vals[ent_class].append(cost)
                    if verbose:
                        print(f"\tClass: {ent_class} → error: {cost}")
        return {
            "parameters": {"w": self.w, "b": self.b},
            "gradients": grads,
            "error_values": error_vals,
        }

    def predict(self,  X):
        x_test_flat = np.array([x.flatten() for x in X])
        result = np.dot(x_test_flat, self.w) + self.b
        predictions = []
        for row in result:
            predictions.append(np.argmax(row))
        
        return predictions

    def validate(self, test_data, true_labels) -> str:
        preds = self.predict(test_data)
        return f"Accuracy for the given data → {100 - np.mean(np.abs(preds - true_labels))*100}"


In [12]:
svm = SVM(X_train,y_train_enc, classes_no=5)
training_result = svm.fit(
    num_iterations=2000,
    lr=0.00029,
    verbose=True,
)

self.w.shape = (16384, 5)
self.b.shape = (1, 5)
Iteration: 0
	Class: 0 → error: 2.0
	Class: 1 → error: 1.9194132288668377
	Class: 2 → error: 1.890978847175176
	Class: 3 → error: 1.9097942714135943
	Class: 4 → error: 1.8959980659711146
Iteration: 100
	Class: 0 → error: 1.6623180092919159
	Class: 1 → error: 1.6612885568350653
	Class: 2 → error: 1.6631550314370354
	Class: 3 → error: 1.6655844117895917
	Class: 4 → error: 1.664943893449542
Iteration: 200
	Class: 0 → error: 1.5175044057731109
	Class: 1 → error: 1.5194915214217835
	Class: 2 → error: 1.5166437767764755
	Class: 3 → error: 1.5187938918703565
	Class: 4 → error: 1.5162035545871062
Iteration: 300
	Class: 0 → error: 1.3974782174060494
	Class: 1 → error: 1.4008407822994857
	Class: 2 → error: 1.397069045643525
	Class: 3 → error: 1.4008128973066196
	Class: 4 → error: 1.3966637394058599
Iteration: 400
	Class: 0 → error: 1.295251826735855
	Class: 1 → error: 1.2998978198878313
	Class: 2 → error: 1.294737712005439
	Class: 3 → error: 1.2994

In [6]:
X_test_flat, y_test_enc_multi = prepare_data(X_test, y_test_enc)
preds = svm.predict(X_test)
size = np.size(preds - y_test_enc)
acc = 0
for i in range(len(preds)):
    if preds[i]== y_test_enc[i]:
        acc += 1

print(
    f"Accuracy for the given data → {(acc/size)*100}%"
)


Accuracy for the given data → 71.42857142857143%
